# 다중 회귀분석 실습

python의 scikit-learn 패키지를 이용해 다중 회귀분석을 직접 실행해보자.

### 데이터 불러오기

Scikit-learn에서는 데이터 분석 연습을 위한 몇 가지의 toy dataset을 제공하고 있다. 그 중 이번 시간에 사용할 데이터셋은 diabetes dataset이다. 

Diabetes dataset은 당뇨병 환자들의 1년간 당뇨병 진행 변화에 대한 데이터이다. 독립 변수는 나이와 성별 등 환자의 정보이며, 예측해야 하는 종속 변수는 당뇨병의 진행 정도이다. 자세한 사항은 [이곳](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset)에서 확인할 수 있다.

결측치가 없고 데이터 정규화(normalization)도 완료되어 있어, 제공된 그대로 분석에 사용하면 된다.

In [ ]:
from sklearn import datasets

In [ ]:
# 데이터 불러오기
data = datasets.load_diabetes()

In [ ]:
# 데이터 확인
for k, v in data.items():
  print(k, ': ', v)

보다 편한 사용을 위해, 독립 변수를 x에 저장하고 종속 변수를 y에 저장하자.

In [ ]:
# 데이터 저장 및 확인
x = data['data']
y = data['target']

print(x.shape, y.shape)

### 데이터 분할하기

수업 시간에 배웠다시피, 적절한 모델을 학습시키기 위해서는 주어진 데이터를 training / validation / test 로 나누는 과정이 필수적이다. 아래처럼  scikit-learn에서 제공하는 [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 함수를 이용하면 편리하게 데이터를 나눌 수 있다.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터 나누기 - 6:2:2 비율
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

print(x_train.shape, x_val.shape, x_test.shape)

### 모델 학습하기

Training data를 이용해 회귀분석 모델을 학습해 보자. 먼저 아래와 같이 필요한 패키지를 import 한다.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

Scikit-learn에서 제공하는 [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html?highlight=linearregression#sklearn.linear_model.LinearRegression) 모델에 대해 살펴보자. 이 객체를 선언할 때 필요한 주요 parameter는 다음과 같다.

1.   fit_intercept: 상수항을 사용할 것인지에 대한 여부. True 일 때 상수항 사용. default=True
2.   normalize: 정규화를 수행할 것인지에 대한 여부. True 일 때 정규화 수행. default=False

이번 실습의 경우는 두 parameter 모두 default 값을 사용하면 된다.


이제 본격적으로 회귀분석 모델을 학습해 보자. Scikit-learn 모델의 학습을 위해 수행해야 할 작업은  

1.   모델 객체 생성
2.   데이터를 이용한 학습

이 전부이다!

In [ ]:
# 회귀분석 객체 생성
mlr = LinearRegression()

# 회귀분석 모델 학습
mlr.fit(x_train, y_train)

추정한 모델의 회귀계수를 확인해 보자. 가장 영향력 있는 변수는?

In [ ]:
# 회귀계수 확인
print(mlr.intercept_)
print(mlr.coef_)

추정한 모델의 설명력을 확인해 보자. 수업 시간에 배운 결정계수 R-square를 계산하려면 우선 training data에 대한 예측값을 알아야 한다.

In [ ]:
# training data 예측값
pred_train = mlr.predict(x_train)

이제 scikit-learn에서 제공하는 r2_score 함수를 이용해 R-square 값을 계산해 보자.

In [ ]:
# training data에 대한 R-square 계산
from sklearn.metrics import r2_score
print(r2_score(y_train, pred_train))

Validation data, test data의 R-sqaure 값도 같은 방법으로 계산해 보자.

In [ ]:
# validation data에 대한 R-square 계산
pred_val = mlr.predict(x_val)
print(r2_score(y_val, pred_val))

In [ ]:
# test data에 대한 R-square 계산
pred_test = mlr.predict(x_test)
print(r2_score(y_test, pred_test))

### 모델 선택하기

수업 시간에 배운 단계적 회귀분석법 중 하나인 전진 선택법을 구현해 보자.

전진 선택법은 상수항만 있는 모형에서 시작해, 설명력을 가장 크게 높일 수 있는 변수부터 하나씩 추가해 나가는 방법이다. 더 이상 설명력의 이득이 없을 때까지 변수를 추가한다. 원래는 F 검정을 기준으로 사용해야 하지만, 간단한 구현을 위해 R-square 로 대체하자. 

공집합에서 시작해, 매 반복마다 남아 있는 변수들 중 "추가했을 때 R-square 가 가장 큰 변수"를 기존의 변수집합에 더해 나가는 방식으로 구현해 보자.

In [ ]:
# 선택된 변수들, R-square 값 및 모델 저장
best_variables = []
best_r2 = 0.
best_model = None

# 남아 있는 변수들
remain_variables = list(range(10))

for round in range(10):
  print(f"============ round {round+1} ============")
  r2_of_this_round = []
  models_of_this_round = []

  for var in remain_variables:
    # 사용될 변수들과 모델
    use_vars = best_variables + [var]
    model = LinearRegression()

    # 지정된 변수만 사용하게끔 데이터 추출
    x_train_small = x_train[:, use_vars]
    x_val_small = x_val[:, use_vars]

    # 지정된 변수로 모델 학습
    model.fit(x_train_small, y_train)
    models_of_this_round.append(model)

    # validation R-square 계산
    r2 = r2_score(y_val, model.predict(x_val_small))
    r2_of_this_round.append(r2)
  
  # R-square 가 높은 모델 선택
  best_r2_of_this_round = np.max(r2_of_this_round)

  # 이전 round와 비교
  if best_r2_of_this_round > best_r2:
    best_var_of_this_round = np.argmax(r2_of_this_round)

    # 변수 추가, R-square 값 및 모델 업데이트
    best_variables.append(remain_variables[best_var_of_this_round])
    best_r2 = best_r2_of_this_round
    best_model = models_of_this_round[best_var_of_this_round]

    # 남은 변수들 중 선택된 변수 제거
    remain_variables.pop(best_var_of_this_round)

    print('best variables updated: ', best_variables)
    print('current best r2: ', best_r2)

  # 더 이상 개선되지 않으면 멈춤  
  else:
    print("no improvement")
    break



print('\n---------------------------------------------------\n')
print('final variables: ', sorted(best_variables))
print('final r2: ', best_r2)
  


In [ ]:
print(r2_score(y_test, best_model.predict(x_test[:, best_variables])))

결정계수 R-square 를 알고 있을 때, 수정 결정계수 adjusted R-square 를 다음과 같이 계산할 수 있다.

In [ ]:
# test data에 대한 adjust R-square 계산
pred_test = mlr.predict(x_test)
test_r2 = r2_score(y_test, pred_test)
test_adj_r2 = 1-(1-test_r2)*(y_test.size-1)/(y_test.size-x_test.shape[1]-1)
print(test_adj_r2)

Adjusted R-square 를 이용하여 후진 제거법 알고리즘을 구현해 보자.

In [ ]:
# 선택된 변수들, R-square 값 및 모델 저장
best_variables = list(range(10))
best_adj_r2 = 0.
best_model = None


# TODO: 선택된 변수들 중 Adjusted R-square의 손실이 가장 적은 변수를 골라 없애기
for round in range(10):
  print(f"============ round {round+1} ============")
  adj_r2_of_this_round = []
  models_of_this_round = []

  for var in best_variables:
    pass

print('---------------------------------------------------')
print('final variables: ', sorted(best_variables))
print('final adj_r2: ', best_adj_r2)

print('---------------------------------------------------')
test_r2=r2_score(y_test, best_model.predict(x_test[:, best_variables]))
test_adj_r2 =1-(1-test_r2)*(y_test.shape[0]-1)/(y_test.shape[0]-len(best_variables)-1) 
print('test adjust R-square: ', test_adj_r2)


### 회귀모형의 검정

수업 시간에 배운 회귀모형의 검정 및 진단을 실행해 보자. 아쉽게도 scikit-learn 에서는 해당 분석을 제공하지 않고 있다. 이를 제공하는 새로운 패키지인 statsmodels 를 사용해 보자.
Statsmodels 패키지에서는 OLS 라는 클래스를 이용하여 회귀분석을 진행한다. 아래와 같은 방식으로 학습을 진행하고 각종 테스트 결과를 확인할 수 있다.

Scikit-learn에서와 다른 점 중 하나는 객체를 생성할 때 데이터를 미리 입력해주어야 한다는 것이고, 또 하나는 상수항을 따로 추가해 주어야 한다는 것이다.

In [ ]:
from statsmodels.regression.linear_model import OLS
import statsmodels.api

In [ ]:
sm_model = OLS(y_train, statsmodels.api.add_constant(x_train))

sm_model = sm_model.fit()
print(sm_model.summary())

다중공선성을 확인할 수 있는 VIF를 계산해 보자.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
for i in range(x_train.shape[1]):
  print(f"VIF of x{i+1}: {variance_inflation_factor(x_train, i):.2f}")

다중공선성이 확인된 변수를 제외한 모델을 학습해 보자.

In [ ]:
# Todo: 다중공선성이 확인된 변수들을 제외하고 회귀분석 학습하고 결과 출력하기(statsmodels 이용)


마지막으로 잔차분석을 위해 잔차 그래프를 그려보자. Matplotlib 패키지의 scatter 함수를 이용해서 아래와 같이 그려볼 수 있다. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.scatter(pred_train, y_train - pred_train)
plt.plot([pred_train.min(), pred_train.max()], [0, 0], '--', color='grey')